In [ ]:
import re
from gensim.models.doc2vec import TaggedDocument
from konlpy.tag import Okt
import pickle

In [ ]:
def clean_str(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern=pattern, repl='', string=text)
    text=text.rstrip('\n')
    return text

def saveP(n):
    fileName=str(n)[:-5]+'.p'
    with open(fileName, 'wb') as file:
        pickle.dump(result, file)


In [ ]:
path='data_all.txt'
fread = open(path, encoding="utf8")

keyId=-1
n=0
result=[]
okt=Okt()


# startPoint : 시작할 지점
# cheak : 저장할 단위 

cheak=100000
startPoint=0

while (1):
    line = fread.readline() #한 줄씩 읽음.
    # [모두 읽으면 while문 종료]
    if not line:
        saveP("9900000")
        break 
        
    # [줄바꿈이면 다음껄로]
    if line=="\n":
        continue

    # [새문서 시작]
    if line[:6]=="</doc>":
        fread.readline()
        fread.readline()
        keyId+=1
    # [새문서 시작2]
    elif line[:4]=='<doc':
        fread.readline()
        keyId+=1

    #그냥문장
    else:
        if startPoint<=n:
            line=clean_str(line)
            lineL=okt.morphs(line)
            taggedL=TaggedDocument(words=lineL, tags=['%d'%keyId])
            result.append(taggedL)
            n+=1
    
            #cheak개마다 저장하고 리셋
            if n%cheak==0:
                saveP(n)
                result=[]
                print(n)
        else:
            n+=1

fread.close()